In [1]:
# import pandas_profiling
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import tkinter as tk
from tkinter import filedialog
import plotly.express as px
import calendar
def my_formatter(x):
    if x < 0:
        return '{:,.0f}'.format(-x)
    return '{:,.0f}'.format(x)
pd.options.display.float_format = my_formatter

In [2]:
category = [calendar.month_name[i] for i in range(1, 13)]

In [3]:
class ProductionSalesReport:
    
    def __init__(self):
        self.path = self.__getFilePath()
        self.df = self.__open_file(self.path)
        
    def __repr__(self):
        return repr(self.df)
    
    def __getFilePath(self):
        '''
        取得開啟檔案路徑
        '''
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename()
        return file_path

    def __open_file(self, path):
        '''
        path: str
        開啟檔案，如果工作表超過一個需要選取
        '''
        df = pd.read_excel(path, None)
        sheets_dict = {}
        if len(df.keys()) != 1:
            for key, value in enumerate(df.keys()):
                print(key + 1, value)
                sheets_dict.setdefault(key + 1, value)
            select_sheet = eval(input('請輸入要開啟的sheet編號：\n'))
            df = df[sheets_dict[select_sheet]]
            return df
        else:
            df = df[list(df.keys())[0]]
            return df

In [4]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='2021預算總表 (revised)')

In [5]:
budget = df

In [ ]:
budget = ProductionSalesReport()

In [ ]:
budget = budget.df

In [6]:
sales = ProductionSalesReport()

C:\Users\kaihsu\Anaconda3\envs\INPAQ_Python_project\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


OverflowError: date value out of range

In [ ]:
sales = sales.df

In [25]:
sales = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\20210610_樞紐分享範本.xlsx", sheet_name='CS OPO')

In [8]:
budget = budget[budget['類型'] == '天線']

In [9]:
budget['類型'] = '預算'

In [10]:
budget['BU'] = budget.apply(lambda x: 'RF2' if x['品名'].startswith('RFDP') else 'RF1', axis=1)

In [11]:
budget = budget.groupby(['類型', '負責業務', 'BU', '預交年份', '預交月份'])[['數量', '本國幣別NTD']].sum().reset_index()

In [26]:
sales['更新日期'] = sales['更新日期'].dt.strftime('%Y-%m-%d')

In [27]:
sales = sales[sales['更新日期'] == sales['更新日期'].max()]

In [28]:
sales['預交月份'] = sales['排定交貨日'].dt.month_name()

In [29]:
sales['預交年份'] = sales['排定交貨日'].dt.year

In [30]:
sales['類型'] = '預估實績'

In [31]:
sales['負責業務'] = sales['負責業務'].map(lambda x: '許凱智' if x in ['楊婉芬', '周彥宏'] else x)

In [33]:
sales = sales[(sales['狀態'].str.contains('出')) & (sales['排定交貨日'].between('2021-01-01', '2021-12-31')) & (sales['BU'] == 'RF1') & (sales['負責業務'].isin(['許凱智', '墨漢雷迪', '鄭里緗']))]

In [34]:
sales = sales.groupby(['類型', '負責業務', 'BU', '預交年份', '預交月份'])[['數量', '集團匯率*金額']].sum().reset_index()

In [35]:
sales.columns = ['類型', '負責業務', 'BU', '預交年份', '預交月份', '數量', '本國幣別NTD']

In [36]:
result = pd.concat([budget, sales], axis=0)

In [ ]:
result['預交月份'] = pd.Categorical(result['預交月份'], categories=category, ordered=True)

In [ ]:
result

In [ ]:
px.histogram(result, x='預交月份', y='本國幣別NTD', color='類型', 
             facet_col='負責業務', barmode='group', category_orders={'預交月份': category}, 
             log_y=True, labels=dict(預交月份='月份', 本國幣別NTD='金額'), title='2021年各月金額')

In [37]:
result.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\預算實績更新總表.xlsx", index=False)

### 未結訂單庫存表

In [ ]:
stock = ProductionSalesReport()

In [ ]:
stock = stock.df

In [ ]:
stock['更新日期'] = stock['更新日期'].dt.strftime('%Y-%m-%d')

In [ ]:
x = stock['更新日期'].unique()[-2:].tolist()

In [ ]:
last_two_week = stock[stock['更新日期'].isin(x)]

In [ ]:
pd.pivot_table(last_two_week, index=['更新日期', '業務人員名稱(NEW)'], columns=['庫存區間'], values=['批次庫存數', '銷售金額(NTD)\n'], aggfunc='sum', fill_value=0)

In [ ]:
df = pd.read_excel(path)

In [ ]:
df['未交量'] = df.apply(lambda x: x['未交量'] * 1000 if x['單位'] == 'KPCS' else x['未交量'], axis=1)

In [ ]:
df['批次庫存數'] = df.apply(lambda x: x['批次庫存數'] * 1000 if x['單位'] == 'KPCS' else x['批次庫存數'], axis=1)

In [ ]:
df['單位'] = df['單位'].map(lambda x: 'PCS' if x == 'KPCS' else x)

In [ ]:
import plotly.express as px

In [ ]:
df.columns

In [ ]:
result = df.groupby(['庫存區間', '業務人員名稱(NEW)', '品名', '送貨客戶名稱'])[['批次庫存數', '未交量', '批次庫存金額NTD', '銷售金額(NTD)\n']].sum().reset_index()

In [ ]:
df.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\20210712\整理過的未結訂單.xlsx", index=False)

## Susan OPO

In [ ]:
path = getFilePath()

In [ ]:
df = pd.read_excel(path, sheet_name='Susan提供的OPO')

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

## CS OPO

In [ ]:
path = getFilePath()
df = pd.read_excel(path, sheet_name='CS OPO')

In [ ]:
df.columns

In [ ]:
df = df.drop_duplicates(subset=['銷售單號', '銷售項次', '客戶訂單', '客戶訂單項次'], keep='last')

In [ ]:
df.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\20210712\整理過的CS OPO.xlsx", index=False)